In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Importing the necessary libraries!
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np
from keras import Sequential
from keras.layers import Dense

Using TensorFlow backend.


# WHITE WINE - PREDICTION OF QUALITY USING NEURAL NETWORKS



In [0]:
# Loading Dataset to a pandas dataframe
df_white = pd.read_csv(
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', delimiter=";")

In [5]:
# Checking if the data has loaded properly!
df_white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [6]:
 print('shape of the dataset:', df_white.shape)

shape of the dataset: (4898, 12)


In [7]:
df_white.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [8]:
# Renaming the column headers without spaces in the names.
df_white.rename(columns={'fixed acidity': 'fixed_acidity','citric acid':'citric_acid',
                         'volatile acidity':'volatile_acidity','residual sugar':'residual_sugar',
                         'free sulfur dioxide':'free_sulfur_dioxide','total sulfur dioxide':'total_sulfur_dioxide'}, inplace=True)
df_white.head(n=5)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [9]:
df_white.quality.unique()

array([6, 5, 7, 8, 4, 3, 9])

**The quality ranges from 1 to 10 - 1 being poor and 10 being the best. We can see the dataset has values 3,4,5,6,7,8,9. We can see we do not have any observations for 1,2 and 10 in our dataset.**

In [10]:
df_white.quality.value_counts().sort_index()

3      20
4     163
5    1457
6    2198
7     880
8     175
9       5
Name: quality, dtype: int64

**We can see the dataset has values 3,4,5,6,7,8,9. We can see we do not have any observations for 1,2 and 10 in our dataset. Hence i have converted the quality to categorical - 
1 - Poor ( for quality ratings < 5)
2 - Average ( for quality ratings 5 & 6)
3 - Best ( for quality ratings > 6)**


In [0]:
df_white.loc[df_white['quality'] < 5, 'quality'] = 1
df_white.loc[df_white['quality'] == 5, 'quality'] = 2
df_white.loc[df_white['quality'] == 6, 'quality'] = 2
df_white.loc[df_white['quality'] > 6, 'quality'] = 3

In [12]:
df_white.quality.unique()

array([2, 3, 1])

In [13]:
df_white.quality.value_counts().sort_index()

1     183
2    3655
3    1060
Name: quality, dtype: int64

In [0]:
np.random.seed(3)

In [0]:
# Number of classes for quality ( Poor, Average, Best)
classes = 3

In [0]:
# Splitting quality into y and keeping the rest of features in X
X = df_white.iloc[:,0:11]
y = df_white.iloc[:,11]

In [17]:
X.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9


In [18]:
y.head()

0    2
1    2
2    2
3    2
4    2
Name: quality, dtype: int64

In [0]:
#standardizing the input feature
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [0]:
# one hot-encoding
y_train = keras.utils.to_categorical(y_train-1, classes)
y_test = keras.utils.to_categorical(y_test-1, classes)

In [22]:
classifier = Sequential()
# First Hidden Layer
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal', input_dim=11))
# Second  Hidden Layer
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
# Output Layer
classifier.add(Dense(3, activation='softmax', kernel_initializer='random_normal'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Compiling the neural network
classifier.compile(optimizer ='adam',loss='categorical_crossentropy', metrics =['accuracy'])

In [27]:
# Fitting the data to the training dataset
classifier.fit(X_train,y_train, batch_size=10, epochs=250)

Epoch 1/250
3428/3428 [==============================] - 0s 98us/step - loss: 0.4720 - acc: 0.7955
Epoch 2/250
3428/3428 [==============================] - 0s 98us/step - loss: 0.4696 - acc: 0.8011
Epoch 3/250
3428/3428 [==============================] - 0s 97us/step - loss: 0.4698 - acc: 0.7926
Epoch 4/250
3428/3428 [==============================] - 0s 97us/step - loss: 0.4704 - acc: 0.7975
Epoch 5/250
3428/3428 [==============================] - 0s 99us/step - loss: 0.4681 - acc: 0.7990
Epoch 6/250
3428/3428 [==============================] - 0s 98us/step - loss: 0.4688 - acc: 0.7970
Epoch 7/250
3428/3428 [==============================] - 0s 99us/step - loss: 0.4706 - acc: 0.7964
Epoch 8/250
3428/3428 [==============================] - 0s 98us/step - loss: 0.4681 - acc: 0.8005
Epoch 9/250
3428/3428 [==============================] - 0s 99us/step - loss: 0.4677 - acc: 0.7958
Epoch 10/250
3428/3428 [==============================] - 0s 98us/step - loss: 0.4679 - acc: 0.7946
Epoch 11/

In [28]:
eval_model=classifier.evaluate(X_train, y_train)
eval_model

3428/3428 [==============================] - 0s 17us/step


[0.4447525202045641, 0.809801633605601]

In [29]:
scores = classifier.evaluate(X_test, y_test)
print("\nAccuracy: %.2f%%" % (scores[1]*100))



1470/1470 [==============================] - 0s 19us/step

Accuracy: 76.33%


# RED WINE - PREDICTION OF QUALITY USING NEURAL NETWORKS


In [0]:
# Loading the dataset for Red wine
df_red = pd.read_csv(
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', delimiter=";")

In [54]:
# Checking if the data has loaded properly!
df_red.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [55]:
 print('shape of the dataset:', df_red.shape)

shape of the dataset: (1599, 12)


In [56]:
df_red.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
fixed acidity           1599 non-null float64
volatile acidity        1599 non-null float64
citric acid             1599 non-null float64
residual sugar          1599 non-null float64
chlorides               1599 non-null float64
free sulfur dioxide     1599 non-null float64
total sulfur dioxide    1599 non-null float64
density                 1599 non-null float64
pH                      1599 non-null float64
sulphates               1599 non-null float64
alcohol                 1599 non-null float64
quality                 1599 non-null int64
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [57]:
# Renaming the column headers without spaces in the names.
df_red.rename(columns={'fixed acidity': 'fixed_acidity','citric acid':'citric_acid',
                         'volatile acidity':'volatile_acidity','residual sugar':'residual_sugar',
                         'free sulfur dioxide':'free_sulfur_dioxide','total sulfur dioxide':'total_sulfur_dioxide'}, inplace=True)
df_red.head(n=5)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [58]:
df_red.quality.unique()

array([5, 6, 7, 4, 8, 3])

In [59]:
df_red.quality.value_counts().sort_index()

3     10
4     53
5    681
6    638
7    199
8     18
Name: quality, dtype: int64

In [0]:
df_red.loc[df_red['quality'] < 5, 'quality'] = 1
df_red.loc[df_red['quality'] == 5, 'quality'] = 2
df_red.loc[df_red['quality'] == 6, 'quality'] = 2
df_red.loc[df_red['quality'] > 6, 'quality'] = 3

In [61]:
df_red.quality.unique()

array([2, 3, 1])

In [62]:
df_white.quality.value_counts().sort_index()

1     183
2    3655
3    1060
Name: quality, dtype: int64

In [0]:
np.random.seed(3)

In [0]:
# Number of classes for quality ( Poor, Average, Best)
classes = 3

In [0]:
# Splitting quality into y and keeping the rest of features in X
X = df_red.iloc[:,0:11]
y = df_red.iloc[:,11]

In [66]:
X.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [67]:
y.head()

0    2
1    2
2    2
3    2
4    2
Name: quality, dtype: int64

In [0]:
#standardizing the input feature
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [0]:
# one hot-encoding
y_train = keras.utils.to_categorical(y_train-1, classes)
y_test = keras.utils.to_categorical(y_test-1, classes)

In [0]:
classifier = Sequential()
# First Hidden Layer
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal', input_dim=11))
# Second  Hidden Layer
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
# Output Layer
classifier.add(Dense(3, activation='softmax', kernel_initializer='random_normal'))

In [0]:
# Compiling the neural network
classifier.compile(optimizer ='adam',loss='categorical_crossentropy', metrics =['accuracy'])

In [74]:
# Fitting the data to the training dataset
classifier.fit(X_train,y_train, batch_size=10, epochs=250)

Epoch 1/250
1119/1119 [==============================] - 0s 130us/step - loss: 0.3440 - acc: 0.8543
Epoch 2/250
1119/1119 [==============================] - 0s 127us/step - loss: 0.3432 - acc: 0.8534
Epoch 3/250
1119/1119 [==============================] - 0s 126us/step - loss: 0.3437 - acc: 0.8525
Epoch 4/250
1119/1119 [==============================] - 0s 132us/step - loss: 0.3435 - acc: 0.8534
Epoch 5/250
1119/1119 [==============================] - 0s 130us/step - loss: 0.3434 - acc: 0.8588
Epoch 6/250
1119/1119 [==============================] - 0s 128us/step - loss: 0.3411 - acc: 0.8597
Epoch 7/250
1119/1119 [==============================] - 0s 145us/step - loss: 0.3422 - acc: 0.8534
Epoch 8/250
1119/1119 [==============================] - 0s 120us/step - loss: 0.3412 - acc: 0.8561
Epoch 9/250
1119/1119 [==============================] - 0s 121us/step - loss: 0.3422 - acc: 0.8561
Epoch 10/250
1119/1119 [==============================] - 0s 129us/step - loss: 0.3415 - acc: 0.8561

In [75]:
eval_model=classifier.evaluate(X_train, y_train)
eval_model

1119/1119 [==============================] - 0s 74us/step


[0.301380371280394, 0.8757819482745391]

In [76]:
scores = classifier.evaluate(X_test, y_test)
print("\nAccuracy: %.2f%%" % (scores[1]*100))



480/480 [==============================] - 0s 31us/step

Accuracy: 85.42%


# CONCLUSION - 


Neural network for 3 categories ( poor, average and best)


Accuracy for white wine dataset - 76.33% ( for 250 epochs)


Accuracy for red wine dataset - 84.42% (for 250 epochs)

